In [ ]:
import os
import requests

if not os.path.exists("competition.py"):
    url = "https://link.teddynote.com/COMPT"
    file_name = "competition.py"
    response = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(response.content)

import competition

# 파일 다운로드
# competition.download_competition_files(project)
competition.download_competition_files(
    "https://link.teddynote.com/ADVHOUSE", use_competition_url=False
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# Data 경로 설정
DATA_DIR = "data"

# 경고 무시
warnings.filterwarnings("ignore")

SEED = 123

## 데이터 로드

In [ ]:
# train 데이터셋 로드 (train.csv)
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))

# test 데이터셋 로드 (test.csv)
test = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

train.head()
test.head()

## 기본 전처리

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data.head()

numeric_columns = all_data.select_dtypes(exclude="object")
numeric_columns

numeric_columns = numeric_columns.fillna(0)
numeric_columns

train_data = numeric_columns[: len(train)]
test_data = numeric_columns[len(train):]

## 데이터셋 분할

In [ ]:
x_train = train_data.drop("SalePrice", axis=1)
y_train = train_data["SalePrice"]

x_test = test_data.drop("SalePrice", axis=1)

## CustomDataset 생성

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


class CustomDataset(Dataset):
    def __init__(self, x, y=None, normalize=True):
        super(CustomDataset, self).__init__()
        self.x = x
        self.y = y

        # 데이터 표준화
        if normalize:
            scaler = StandardScaler()
            self.x = pd.DataFrame(scaler.fit_transform(self.x))

        # 텐서 변환
        self.x = torch.tensor(self.x.values).float()

        if y is not None:
            self.y = torch.tensor(self.y).float()

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        if self.y is not None:
            y = self.y[idx]
            return x, y
        else:
            return x

1) import
import torch: PyTorch 라이브러리를 가져옵니다.
from torch.utils.data import Dataset: PyTorch의 사용자 정의 데이터 세트에 대한 기본 클래스인 PyTorch에서 Dataset 클래스를 가져옵니다.
from sklearn.preprocessing import StandardScaler: 데이터 표준화에 사용되는 scikit-learn에서 StandardScaler를 가져옵니다.

2) CustomDataset 클래스:

이 클래스는 torch.utils.data.Dataset에서 상속되는 사용자 정의 데이터세트 클래스로, PyTorch의 데이터 로딩 유틸리티와 호환됩니다.

3) 생성자 메서드(__init__):

def __init__(self, x, y=None, Normalize=True):: CustomDataset 클래스를 초기화합니다. 'x'(기능) 및 'y'(대상)를 입력으로 사용하며, 여기서 'y'는 선택 사항입니다.
super(CustomDataset, self).__init__(): 상위 클래스(Dataset)의 생성자를 호출합니다.
self.x = x 및 self.y = y: 입력 특성과 대상을 인스턴스 변수에 할당합니다.
데이터 표준화:
normalize가 True인 경우 scikit-learn의 StandardScaler를 사용하여 입력 기능을 표준화합니다.

텐서 변환:
표준화된 기능(self.x)을 float 유형의 PyTorch 텐서로 변환합니다.
y가 제공되면(None 아님) 대상(self.y)도 float 유형의 PyTorch 텐서로 변환됩니다.


4) __len__ 방법:


def __len__(self):: 데이터세트의 길이, 즉 샘플 수를 정의합니다. 입력 특성(self.x)의 길이를 반환합니다.

5) __getitem__ 메소드:


def __getitem__(self, idx):: 인덱스 idx가 지정된 데이터 세트에서 샘플을 검색합니다.
x = self.x[idx]: idx 인덱스에 해당하는 특징을 검색합니다.
y가 제공되는 경우(None 아님):
y = self.y[idx]: idx 인덱스에 해당하는 대상을 검색합니다.
샘플의 특징과 대상을 나타내는 튜플 (x, y)를 반환합니다.

y를 제공하지 않으면(None) 기능만 반환합니다.